In [1]:
from keras import datasets, layers,utils,Sequential
import numpy as np
from keras.src.layers import Dense
from keras_tuner.tuners import RandomSearch,Hyperband,BayesianOptimization

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [3]:
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000, 784)
x_train=x_train/255
x_test=x_test/255
y_train=utils.to_categorical(y_train,10)
y_test=utils.to_categorical(y_test,10)

In [4]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation_choice', ['relu', 'tanh', 'sigmoid','elu','selu'])
    model.add(Dense(units=hp.Int('units_input', min_value=512, max_value=1024, step=32), activation=activation_choice,input_dim=784))
    model.add(Dense(units=hp.Int('units_hidden',min_value=128, max_value=600, step=32), activation=activation_choice))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer=hp.Choice('optimizer',values=['adam','rmsprop','SGD']), metrics=['accuracy'])
    return model

In [5]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=80,directory='test_directory')
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation_choice (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
